<a href="https://colab.research.google.com/github/divassya/BigDataAnalysis/blob/main/AssiyaKaratay_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Spark installation on Colab
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
# !tar xf spark-3.0.1-bin-hadoop3.2.tgz
# !pip install -q findspark
# !rm -rf spark-3.0.1-bin-hadoop3.2.tgz

!pip install --ignore-installed -q pyspark==3.1.2 

In [3]:
# Set JAVA_HOME and SPARK_HOME
import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = "spark-3.0.1-bin-hadoop3.2"

# import findspark
# findspark.init("spark-3.0.1-bin-hadoop3.2")# SPARK_HOME


import sys
import requests
from operator import add

from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

from pyspark.sql.types import *
from pyspark.sql import functions as func
from pyspark.sql.functions import *

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [3]:
import bz2

In [4]:
# get the data
path = "./"
filename = "taxi-data-sorted-small.csv.bz2"
# filename = sys.argv[1]
# with bz2.open(filename, 'rt', encoding='utf-8') as f:
lines = sc.textFile(filename)
taxilines = lines.map(lambda x: x.split(','))

In [6]:
# Exception handling and removing wrong data lines
def isfloat(value):
  try:
    float(value)
    return True
  except:
    return False

# remove lines if they do not have 17 values
def correctRows(p):
  if (len(p) == 17):
    if (isfloat(p[5]) and isfloat(p[11])):
      if (float(p[5])!=0 and float(p[11])!=0):
        return p

#cleaning up data
taxilinesCorrected = taxilines.filter(correctRows)

In [7]:
# the top ten taxis that have had the largest number of drivers. 
# p[0] is the medallion of the taxi and p[1] is hack license of the driver

# 1. get a pair of distinct medallions and drivers
TaxiDriver = taxilinesCorrected.map(lambda p: (p[0], p[1]))
distinctTaxiDriver = TaxiDriver.distinct()
# 2. count number of drivers for each taxi 
countDriversPerTaxi = distinctTaxiDriver.map(lambda p: (p[0], 1)).reduceByKey(lambda a,b: a+b)

#Get top 10 taxis and show them as (medallion,number of drivers) 
top10Taxis = sc.parallelize(countDriversPerTaxi.top(10, lambda x:x[1]))
#Save to text file
output_file = 'top10Taxis.txt'
output_file = sys.argv[2]
top10Taxis.saveAsTextFile(output_file)

sc.stop()  

Py4JJavaError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')